# Tower of Hanoi

A mathematical game or puzzle that consists of three rods and a number of disks of different sizes, which can slide onto any rod.

The objective is to move the entire stack into another rod under the following constraints:

1. Only one disk can be moved at a time
2. Each move consists of taking the upper disk from one of the stacks and placing it on top of another stack.
3. No disk may be placed on top of a smaller disk.


With 3 disks, the puzzle can be solved in 7 moves. The minimal number of moves required to solve a Tower of Hanoi is $2^n - 1$, where $n$ is the number of disks.

There are few ways to solve this:

1. Recursion
2. Q-learning

We will be focusing on the latter.

## Q-learning

*Q-learning* is a *model-free* reinforcement learning technique. It works by learning an action-value function that ultimately gives the expected utility of taking a given action in a given state and following the optimal policy thereafter.

In [11]:
import numpy as np

class Pole:
    def __init__(self, discs):
        self.discs = discs
        return
    
    def can_add(self, value):
        if (len(self.discs) == 0):
            return True
        last_disc = self.discs[-1]
        return last_disc > value
    
    
    def add(self, disc):
        self.discs.append(disc)
        return

    def can_remove(self):
        return len(self.discs) > 0

    def remove(self):
        return self.discs.pop()

    def __str__(self):
        return "{}".format(self.discs)

# Create a new pole, the larger number indicates larger disc size
pole = Pole([3,2,1])

# Check if the disc can be removed
print(pole.can_remove())
disc = pole.remove()

print(disc)

# Check if the disc can be added
print(pole.can_add(disc))
pole.add(disc)

print(pole.discs)

True
1
True
[3, 2, 1]


In [18]:
from itertools import permutations
from collections import defaultdict
import random
import copy

class Hanoi:
    def __init__(self, *poles):
        self.poles = [pole for pole in poles]
        # Our reward matrix.For simplicity, we know that we won't 
        # be making invalid moves and hence we only define 
        # the final positive reward (when all
        # the discs is in the last pole, ccc)
        self.r = defaultdict()
        self.r['acc'] = {}
        self.r['bcc'] = {}

        # Here we define the state that leads to action
        # See Wikipedia Tower of Hanoi
        self.r['acc']['ccc'] = 100
        self.r['bcc']['ccc'] = 100
        
        
        self.q = defaultdict()
        for p in self.permutations_with_repetition('abc'):
            self.q[p] = {}
            for q in self.permutations_with_repetition('abc'):
                self.q[p][q] = 0
        
    def permutations_with_repetition(self, s):
        base = len(s)
        for n in range(base**base):
            yield ''.join(s[n // base**(base-d-1) % base] for d in range(base))

    def add (self, pole, value):
        self.poles[pole].add(value)
        return
    
    def can_add(self, pole, disc):
        return self.poles[pole].can_add(disc)

    def can_remove(self, pole):
        return self.poles[pole].can_remove()

    def remove(self, pole):
        return self.poles[pole].remove()
    
    def pprint(self):
        [ print('{}: {}'.format(i, disc)) for i, disc in enumerate(self.poles) ]

    def get_combination(self):
        '''
            Returns the combination of the discs in abc format.
            For example, given the following:
                x = [[3], [2, 1], []]
            This function will return 'bba'.
            a, b and c refers to the first, second and third pole. 
            In words, it states that disc 1 and 2 is in pole b, while
            disc 3 is in pole a.

                x = [[3,2,1], [], []]
            This will return 'aaa', which means that all discs is in 
            the first pole.
        '''
        state = [0] * 3
        poles = [pole.discs for pole in self.poles]
        for i, v in enumerate(poles):
            if 1 in poles[i]:
                state[0] = 'abc'[i]
            if 2 in poles[i]:
                state[1] = 'abc'[i]
            if 3 in poles[i]:
                state[2] = 'abc'[i]
        return ''.join(state)

    def play(self):
        '''
            Play the game, compute the Q(s,a)
        '''
        # Iterate for different episodes
        original = copy.deepcopy(self.poles)
        for i in range(100):
            self.poles = copy.deepcopy(original)
            # Keep playing until the last pole has three discs
            while len(self.poles[-1].discs) != 3:
                disc = random.randint(0, 2)

                # Get the initial state
                state = self.get_combination()
                if self.can_remove(disc):
                    disc = self.remove(disc)
                    target = random.randint(0,2)

                    while not self.can_add(target, disc):
                        target = random.randint(0,2)
                    self.add(target, disc)

                    # Get the action
                    action = self.get_combination()

                    # Our Q-learning
                    if self.q.get(state) == None:
                        self.q[state] = {}

                    if self.q[state].get(action) == None:
                        self.q[state][action] = 0.0

                    if self.q.get(action) == None:
                        self.q[action] = {}

                    if self.r.get(state) == None:
                        self.r[state] = {}

                    if self.r[state].get(action) == None:
                        self.r[state][action] = 0.0

                    # Future actions score
                    reward = 0
                    scores = [score for (_, score) in self.q[action].items()]
                    if (len(scores)) > 0:
                       reward = max(scores)
                    self.q[state][action] = self.r[state][action] + 0.8 * reward

pole1 = Pole([3,2,1])
pole2 = Pole([])
pole3 = Pole([])
hanoi = Hanoi(pole1, pole2, pole3)

print('initial_state:')
hanoi.pprint()

print('play:', hanoi.play())

print('final_state:')
hanoi.pprint()


print('looking for optimum route')
best_move = 'aaa'
print('starting with {}'.format(best_move))
while best_move != 'ccc':
    best_score = max([score for (_,  score) in hanoi.q[best_move].items()])
    best_move = [move for (move, score) in hanoi.q[best_move].items() if score == best_score][0]
    print('best_move', best_move)

# print('\n'.join(["{0:^9s}".format('=' * (i*2 + 1)) for i in range(3)]))

initial_state:
0: [3, 2, 1]
1: []
2: []
play: None
final_state:
0: []
1: []
2: [3, 2, 1]
looking for optimum route
starting with aaa
best_move caa
best_move cba
best_move bba
best_move bbc
best_move abc
best_move acc
best_move ccc
